### Imports

In [31]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *

import warnings
warnings.filterwarnings('ignore')

ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline
%matplotlib notebook

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


from sklearn_pandas import *

import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.preprocessing import *
from sklearn.cross_validation import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.datasets import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.tree import *
from sklearn.grid_search import *
from sklearn.base import *
from sklearn.datasets.twenty_newsgroups import *
from sklearn.decomposition import *
from sklearn.feature_extraction import *
from sklearn.metrics import *
# n.download()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### R script adding new features to dataset 

In [2]:
# http://www.mango-solutions.com/wp/2015/10/integrating-python-and-r-part-ii-executing-r-from-python-and-vice-versa/
command = 'Rscript'
path2script = 'lexical_features.R'

# Build subprocess command
cmd = [command, path2script]

subprocess.check_output(cmd, universal_newlines=True)

NameError: name 'subprocess' is not defined

In [3]:
data_n = pd.read_csv("../raw_data/data_n.csv", parse_dates=True, infer_datetime_format=True)
data_n.head()

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,4.400,0,0,0,0.700,2,0,0
1,2,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.615,0,0,0,0.577,10,1,0
2,3,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.500,0,2,-2,0.509,14,1,0
3,4,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3,3.15,3.25,4.4,n,y,...,0,0,3.667,0,0,0,0.611,4,0,0
4,5,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3,3.15,3.25,4.4,n,y,...,0,0,2.333,0,0,0,0.857,1,0,0


### Replace 'y' and 'n' with 1 & 0

In [4]:
# http://stackoverflow.com/a/17702781
d = {'n': 0, 'y': 1}
data_n = data_n.replace(d)
data_n.head(n=1)

,Unnamed: 0,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,...,Number_of_Semicolons,Number_of_Colons,Average_Word_Length,POS_sentiment,NEG_sentiment,OverAll_sentiment,Lexical_Diversity,Number_of_FunctionalWords,Number_of_Pronouns,Number_of_PROPNAMEs
0,1,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,0,1,...,0,0,4.4,0,0,0,0.7,2,0,0


In [ ]:
#norm:http://blog.yhat.com/posts/predicting-customer-churn-with-sklearn.html
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
# to_drop = ['State','Area Code','Phone','Churn?']
# churn_feat_space = data_n.drop(to_drop,axis=1)

### Split stra. k-fold + selection of columns

In [5]:
# neu = data_n[["#AUTHID","STATUS","cNEU"]]
# train_neu, test_neu, y_train, y_test = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify= neu["cNEU"])
# train_neu, test_neu, y_train, y_test 

In [6]:
neu = data_n[["#AUTHID","STATUS","cNEU", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
ext = data_n[["#AUTHID","STATUS","cEXT", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
agr = data_n[["#AUTHID","STATUS","cAGR", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
con = data_n[["#AUTHID","STATUS","cCON", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]
opn = data_n[["#AUTHID","STATUS","cOPN", "StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]]

In [38]:
train_feat_neu, test_feat_neu, train_class_neu, test_class_neu = sk.cross_validation.train_test_split(neu, neu["cNEU"], train_size = 0.66, stratify = neu["cNEU"])
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66, stratify= ext["cEXT"])
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66, stratify= agr["cAGR"])
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66, stratify= con["cCON"])
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66, stratify= opn["cOPN"])

### Train + Test

In [48]:
mku = sk.pipeline.make_pipeline(SGDClassifier(), LinearSVC(),
                               # BernoulliNB(), #MultinomialNB()
                               # KNeighborsClassifier(),
              RandomForestClassifier(n_jobs=-1),AdaBoostClassifier()) 

In [49]:
mku.fit(train_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]], train_class_neu)

Pipeline(steps=[('sgdclassifier', SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=...m='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None))])

In [52]:
testar = mku.predict(test_feat_neu[["StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]])

In [43]:
# predicted = mku.predict(test_neu[["StringLength", 
#                   "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
#                   "Number_of_Colons", "Average_Word_Length", "Lexical_Diversity"]])

In [10]:
new_fea = data_n[["STATUS","cCON","cOPN","StringLength", 
                  "Number_of_Words", "Number_of_Dots", "Number_of_Commas", "Number_of_Semicolons", 
                  "Number_of_Colons", "Avarage_Word_Lenght", "Lexical_Diversity"]]
new_fea.head(n=1)

KeyError: "['Avarage_Word_Lenght'] not in index"

### Confusion Matrix

In [53]:
cm = ConfusionMatrix(test_class_neu, testar)
cm.print_stats()

Confusion Matrix:

Predicted   0.0  1.0  __all__
Actual                       
0.0         621    1      622
1.0         534    2      536
__all__    1155    3     1158


Overall Statistics:

Accuracy: 0.537996545769
95% CI: (0.50878419260463481, 0.56701544430656814)
No Information Rate: ToDo
P-Value [Acc > NIR]: 1.0
Kappa: 0.00228039153163
Mcnemar's Test P-Value: ToDo


Class Statistics:

Classes                                         0           1
Population                                   1158        1158
P: Condition positive                         622         536
N: Condition negative                         536         622
Test outcome positive                        1155           3
Test outcome negative                           3        1155
TP: True Positive                             621           2
TN: True Negative                               2         621
FP: False Positive                            534           1
FN: False Negative                              1